In [1]:
from importlib import reload

In [2]:
from datetime import datetime, timedelta

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

from IPython.display import Image, display

from dowhy import CausalModel

In [3]:
pd.set_option("mode.chained_assignment", None)

In [4]:
import futil, load_util

In [5]:
wdi_df, series_df = load_util.load_wdi()
project_df = load_util.load_projects()
wb_df = load_util.extract_wb_projects(project_df)

In [6]:
odi_and_gdp_indicators = [
    'DT.ODA.ODAT.GN.ZS',
    'NY.GDP.MKTP.CD' # divide project size at time
]

education_expense_indicators = [
    'SE.XPD.PRIM.ZS',
    'SE.XPD.SECO.ZS',
    'SE.XPD.TERT.ZS'
]

education_outcome_indicators = [
    'SE.PRM.NENR', 
    'SE.PRM.CMPT.ZS', 
    'SE.PRM.PRSL.ZS'
]

## Constructing time gaps

In [7]:
sector_aggs = project_df.groupby('sector').agg(
    number_projects=('ppd_project_id', 'nunique'),
    earliest_year=('start_dt', 'min'),
    latest_project=('start_dt', 'max'),
    number_donors=('donor_name', 'nunique'),
    avg_rating=('six_overall_rating', 'mean'),
    avg_duration=('project_duration', 'mean')
)

In [8]:
feature_cols = [
    'country_code', 
    'donor_name', 
    'aiddata_sectorname', 
    'six_overall_rating', 
    'start_date', 
    'completion_date', 
    'project_duration',
    'sector'
]

pdf = load_util.narrow_convert_project_data(project_df, feature_cols)

total_countries = pdf.country_code.nunique()
earliest_year = pdf.start_year.min()
latest_year = pdf.start_year.max()
year_range = range(int(earliest_year), int(latest_year))

In [11]:
futil.calc_sector_controls(pdf, 'Education', 1990, year_range)

## First approach: binary treatment, simple, education

### Assemble education indicator frame

In [ ]:
# for each country-year, calculate mean growth in education indicators at year + lag

In [ ]:
df, tdf = futil.assemble_sectoral_df(
    pdf, 
    education_outcome_indicators,
    year_range,
    interpolate_limit=2
)

In [ ]:
ug_df = tdf[(tdf['Country Code'] == 'UGA') & (tdf['Indicator Code'] == 'SE.PRM.PRSL.ZS')]
ug_df = ug_df.set_index('Year')

In [ ]:
ug_df.tail()

In [ ]:
ug_df['value_interp'] = ug_df['value'].interpolate(method='nearest', order=5)
ug_df[ug_df.value_interp > 0].plot.line(y=['value_interp', 'value'])

In [ ]:
# third, obtain the average outcome growth, for the response, and whether a project was completed in a given year
# for the treatment (so far)

In [ ]:
df.head()

In [ ]:
def is_project_complete_year(year, country, sp_df):
    country_proj = sp_df[sp_df['country_code'] == country].end_year.to_list()
    return bool(year in country_proj)

In [ ]:
health_proj = project_df[project_df.sector == 'Health']

In [ ]:
print(len(health_proj))

In [ ]:
start_years = health_proj.groupby('country_code')['start_year'].apply(set).to_dict()

In [ ]:
df['project_completed_year'] = df.apply(
    lambda row: row['country'] in start_years and row['year'] in start_years[row['country']], 
axis=1)

In [ ]:
df['project_completed_year'] = df.apply(lambda row: is_project_complete_year(row['year'], row['country'], health_proj), axis=1)

In [ ]:
country_proj = sp_df[sp_df['country_code']]

In [ ]:
df = construct_sector_lag_table(df, tdf, pdf, education_outcome_indicators, 'Education')

In [ ]:
sp = project_df[project_df.sector == 'Education']

In [ ]:
df.columns

In [ ]:
df['project_completed_year'] = df.apply(lambda row: row['year'] in sp[sp['country_code'] == row['country']].end_year.to_list(), axis=1)

In [ ]:
print('Total projects: ', len(pdf))
print('Years with sufficient coverage on lag: ', len(df[(df.education_lag_4_count > 0)]))

In [ ]:
df.project_completed_year.value_counts()

In [ ]:
df.head()

In [ ]:
# project_df.head()

In [ ]:
# remove: years within the influence of a project, i.e., during its initation and up to the lag period

In [ ]:
# then: add in the various confounders, treatments, controlsc

In [ ]:
def find_latest_govt_rating(country, year):
    country_proj = project_df[
        (project_df['country_code'] == country) 
        & (project_df['start_year'] <= year)
        & (project_df['wb_government_partner_rating'].notna())]
    if len(country_proj) == 0:
        return np.nan
    else:
        proj_year = country_proj[country_proj.start_year == country_proj.start_year.max()]
        return proj_year['wb_government_partner_rating'].sort_values(ascending=False).iloc[0]
    
#     return country_proj[['start_year', 'wb_government_partner_rating']]

In [ ]:
if 'most_recent_govt_rating' not in df:
    df['most_recent_govt_rating'] = df.apply(lambda row: find_latest_govt_rating(row['country'], row['year']), axis=1)

In [ ]:
df.most_recent_govt_rating.value_counts()

In [ ]:
df.to_csv('../data/education_df.csv', index=False)

# Execute do-why

In [ ]:
df['sector_delta_prior'] = df['education_lag_-4_growth']
df['sector_delta_post'] = df['education_lag_4_growth']

In [ ]:
sectoral_effect_graph_binary = """digraph { 
    "Govt-Quality"[latent];
    project_completed_year [exposure, label="project_completed"];
    sector_delta_prior [adjusted, label="sector_outcome_growth_prior"];
    sector_delta_post [outcome, label=sector_outcome_growth_lagged];
    most_recent_govt_rating [label="WBGovtRating"];
    "Govt-Quality" -> project_completed_year;
    "Govt-Quality" -> sector_delta_prior;
    "Govt-Quality" -> most_recent_govt_rating;
    project_completed_year -> sector_delta_post;
    sector_delta_prior -> sector_delta_post;
    most_recent_govt_rating -> project_completed_year;
    }"""

In [ ]:
causal_columns = ['sector_delta_prior', 'most_recent_govt_rating', 'project_completed_year', 'sector_delta_post']

In [ ]:
causal_df = df[causal_columns]
pre_drop_N = len(causal_df)
causal_df = causal_df.dropna()
post_drop_N = len(causal_df)

print("N pre NA drop: ", pre_drop_N, " and post: ", post_drop_N)

In [ ]:
causal_df.project_completed_year.value_counts()

In [ ]:
model = CausalModel(
    data=causal_df,
    graph=sectoral_effect_graph_binary.replace("\n", " "),
    treatment="project_completed_year",
    outcome="sector_delta_post"
)

model.view_model()
display(Image(filename="causal_model.png"))

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
estimate = model.estimate_effect(identified_estimand, method_name="backdoor.linear_regression",
                                 target_units="ate", test_significance=True)
print(estimate)

In [ ]:
causal_df.head()

In [ ]:
estimator_list = ["backdoor.propensity_score_matching", "backdoor.propensity_score_weighting", "backdoor.causalml.inference.meta.LRSRegressor"]
method_params= [ None, None, { "init_params":{} } ]

In [ ]:
refuter_list = ["bootstrap_refuter", "data_subset_refuter"]

In [ ]:
# for estimator, method_params in zip(estimator_list, method_params):
#     estimate = model.estimate_effect(identified_estimand, method_name=estimator, method_params=method_params)

In [ ]:
refuter = model.refute_estimate(identified_estimand, estimate, method_name="bootstrap_refuter")

In [ ]:
print(refuter)

In [ ]:
run_refuter = lambda refuter_name, refuter_params: model.refute_estimate(identified_estimand, estimate, method_name=refuter_name, **refuter_params)

In [ ]:
# c_cause_refuter = model.refute_estimate(identified_estimand, estimate, method_name="random_common_cause")
c_cause_refuter = run_refuter("random_common_cause", {})

In [ ]:
print(c_cause_refuter)

In [ ]:
placebo_refuter = run_refuter("placebo_treatment_refuter", dict(placebo_type="permuter"))

In [ ]:
print(placebo_refuter)

In [ ]:
sample_refuter = run_refuter("data_subset_refuter", dict(subset_fraction=0.9))

In [ ]:
print(sample_refuter)

In [ ]:
# Note: took a bit of digging, but the "p value" here relates to a null hypothesis that 
# the estimate obtained is part of the general set of distributions, i.e., we want this to hold
# (since it suggests that the family of estimators is not significantly affected by subsampling)

In [ ]:
estimate.interpret(method_name="textual_effect_interpreter")

In [ ]:
bootstrap_refuter = run_refuter("bootstrap_refuter", {})

In [ ]:
print(bootstrap_refuter)

In [ ]:
# cribbed for now from example notebook
dummy_args = {'num_simulations':5,'transformation_list': [('random_forest',{'n_estimators':100, 'max_depth':6})], 'true_causal_effect':(lambda x:0.5)}

In [ ]:
dummy_refuter = run_refuter("dummy_outcome_refuter", dummy_args)

In [ ]:
print(dummy_refuter[0])

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LassoCV

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# cribbed for now
dml_params = { 
    "init_params": { 
        'model_y':GradientBoostingRegressor(),
        'model_t': GradientBoostingRegressor(),
        "model_final":LassoCV(fit_intercept=False),
        'featurizer':PolynomialFeatures(degree=1, include_bias=True)
    },
    "fit_params": {}
}

In [ ]:
estimate = model.estimate_effect(identified_estimand, method_name="backdoor.econml.dml.DML", method_params=dml_params,
                                 target_units="ate", test_significance=True)
print(estimate)

In [ ]:
print(estimate)